In [2]:
from mesa.batchrunner import BatchRunner, BatchRunnerMP
from mesa.time import RandomActivation
from model import *
from customizeTime import *
import pickle
import numpy as np
import uuid

from customizeTime import *
import env_settings

TYPE_MAP = {0:'vert',
            1:'horz'}

settings.MUTATION_PRO = 0.01

class A_2(GenAgent):
    def muta_genetic_info(self):
        # no need for mutation
        pass
    
        
class M_2(GenModel):
    def get_agent_config(self):
        return A_2
    
    def init_env(self):
        #no influence
        return [self.env_val] * 5000
    
    def get_schedule(self):
        return RandomActivationMutationWithConstrain
    #
    def __init__(self, N, width, height, specific_kargs):
        #mut_var, init_var, env_val
        self.model_name = 'M2'
        
        init_ratio = specific_kargs['init_ratio']
        self.env_val = specific_kargs['env_val']
        
        
        settings.INIT_MEAN = specific_kargs['gene_mean']
        settings.INIT_SD = specific_kargs['gene_sd']
        settings.INIT_LOW = settings.INIT_MEAN - 1 * 2 * settings.INIT_SD
        settings.INIT_HIGH = settings.INIT_MEAN + 1 * 2 * settings.INIT_SD

        settings.POPU_MUTA_SD = specific_kargs['muta_sd']
        settings.POPU_MUTA_LOW = -2 * settings.POPU_MUTA_SD
        settings.POPU_MUTA_HIGH = 2 * settings.POPU_MUTA_SD
        
        self.specific_kargs_str = '_'.join([str(i) for i in (self.env_val, settings.INIT_SD ,settings.POPU_MUTA_SD)])
        super().__init__(N, width, height, init_ratio)
        

    
    def model_level_record_init(self):
        data_file_name = check_path("./res/")
        data_file_name = check_path(data_file_name + self.model_name)
        data_file_name = check_path(data_file_name + "/" + self.specific_kargs_str)
        data_file_name = data_file_name + '/' + str(uuid.uuid1())
        print('data in file ', data_file_name)
        self.data_file = open(data_file_name, 'w')
        self.data_file.write('#' + '\t'.join(
            ('step',
             TYPE_MAP[0], TYPE_MAP[0] + '_beta', TYPE_MAP[0] + '_gene', 'var',  
             TYPE_MAP[1], TYPE_MAP[1] + '_beta', TYPE_MAP[1] + '_gene', 'var'))+ '\n')
       
    def get_popu_beta(self, gen_type):
        return np.mean([agent.self_gen_cof for agent in self.schedule.agents if agent.gen_type == gen_type])
    
    def get_gene_mean(self, gen_type):
        return [np.mean(agent.gen_info) for agent in self.schedule.agents if agent.gen_type == gen_type]
    
    def get_popu_gene_mean(self, gene_mean):
        return np.mean(gene_mean)

    def get_popu_gene_var(self, gene_mean):
        return np.var(gene_mean)

    def collect_data(self):
        ver_mean = self.get_gene_mean(0)
        hor_mean = self.get_gene_mean(1)
        return (self.schedule.steps,
                self.get_ver_popu_size(), self.get_popu_beta(0), self.get_popu_gene_mean(ver_mean), self.get_popu_gene_var(ver_mean),
                self.get_her_popu_size(), self.get_popu_beta(1), self.get_popu_gene_mean(hor_mean), self.get_popu_gene_var(hor_mean))
        
        
    def step(self):
        
        data = self.collect_data()
        #print(data)
        sys.stdout.write('\t'.join([str(d) for d in data]) + '\n')
        #if self.schedule.steps % 10 == 0: 
        self.data_file.flush()
        super().step()
        
        


In [3]:
import sys

#0.01, 0.005, 0.001
sys.argv = 'm2 1 1'.split(' ')

env_val, muta_sd = float(sys.argv[1]), float(sys.argv[2])

fixed_params = {'N':2000, 'width':100, 'height':100}

variable_params = {'specific_kargs': [{'env_val': env_val , 'init_ratio':0.5, 'gene_mean':0.5, 'gene_sd':muta_sd, 'muta_sd':muta_sd}]}

batch_run = BatchRunnerMP(
    M_2, 1,
    variable_parameters=variable_params,
    fixed_parameters=fixed_params,
    iterations=1,
    max_steps=2000,
)
batch_run.run_all()

0it [00:00, ?it/s]

data in file  ./res/M2/1.0_1.0_1.0/c612923e-d1bc-11e9-9769-f2189850a77c
0	991	0.7290534623842985	0.4980944211790449	0.019037327564103573	1009	0.7408649293584868	0.5079339952032912	0.01700611756491851
0 population size:  2000
1	1281	0.7510180871485448	0.5256992238426386	0.024689130077207532	1361	0.7572527333890334	0.525374514855372	0.017784953846768516
1 population size:  2642
2	1609	0.7727346347468089	0.5551249174478022	0.029336350431470946	1725	0.7720725211167061	0.5403360161338501	0.016956737574118357
2 population size:  3334
3	2011	0.7954968766543249	0.5845726228868915	0.03232783395707735	2147	0.7833662927623188	0.5538585004165059	0.017653827959796486
3 population size:  4158
4	2452	0.8199553645236999	0.6184934103776087	0.034423997821650176	2588	0.7952225522732052	0.5674846075480929	0.017662577787043002
4 population size:  5040
5	2945	0.8398278809758647	0.6507996464346272	0.038362895450448777	2968	0.8052186035151997	0.5793982736504689	0.017725477220732724
5 population size:  5913
6	

Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/site-packages/multiprocess/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.7/site-packages/pathos/helpers/mp_helper.py", line 15, in <lambda>
    func = lambda args: f(*args)
  File "/usr/local/lib/python3.7/site-packages/mesa/batchrunner.py", line 153, in run_iteration
    self.run_model(model)
  File "/usr/local/lib/python3.7/site-packages/mesa/batchrunner.py", line 178, in run_model
    model.step()
  File "<ipython-input-2-9cfa5184d967>", line 95, in step
    super().step()
  File "/Users/zhuliu/projects/ye/selection/model.py", line 151, in step
    self.schedule.step()
  File "/Users/zhuliu/pr

KeyboardInterrupt: 

In [4]:
import math
math.sqrt(0.1)

0.31622776601683794